<a href="https://colab.research.google.com/github/haydenmclemore-fresnost/math120_final_project/blob/main/MATH120_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# California Water Quality Analysis – Final Project
This notebook analyzes selected water quality measurements from the California Department of Water Resources (DWR).  
The goal is to clean the data, merge it with station information, and answer a few basic questions using tools from class.


## Environment Setup


In [35]:
from google.colab import drive
drive.mount('/content/drive')

DATA_DIR = "/content/drive/MyDrive/MATH120_Final_Project"
stations_path = f"{DATA_DIR}/stations.csv"
field_path = f"{DATA_DIR}/field_results.csv"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Import Libraries


In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option("display.max_columns", 50)

## Load Data


In [37]:
stations = pd.read_csv(stations_path)
field = pd.read_csv(field_path, nrows=200000)  # sample for speed

stations.head(), field.head()


/tmp/ipython-input-781344394.py:2: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  field = pd.read_csv(field_path, nrows=200000)  # sample for speed


(   _id  station_id     station_name         full_station_name station_number station_type  latitude  longitude  \
 0    1       47023  Frank Sieferman  Frank Sieferman New Well  01N01E01A001M  Groundwater   33.0000  -121.0000   
 1    2        6134    01N01E33K001M             01N01E33K001M  01N01E33K001M  Groundwater   37.8868  -121.8682   
 2    3        6135    01N01W03N001M             01N01W03N001M  01N01W03N001M  Groundwater   37.9583  -121.9669   
 3    4        6136    01N01W04A001M             01N01W04A001M  01N01W04A001M  Groundwater   37.9663  -121.9734   
 4    5        6137    01N01W07K001M             01N01W07K001M  01N01W07K001M  Groundwater   37.9460  -122.0155   
 
     county_name  sample_count      sample_date_min      sample_date_max  
 0          Yolo             1  2008-06-23T15:00:00  2008-06-23T15:00:00  
 1  Contra Costa             1  1981-05-20T13:45:00  1981-05-20T13:45:00  
 2  Contra Costa             9  1974-07-23T15:15:00  1990-08-16T12:15:00  
 3  Cont

## Clean and Standardize Columns


In [38]:
# We don't really need to rename anything except create a 'result' column

# Make a numeric result column from fdr_result
field["result"] = field["fdr_result"]

# Ensure sample_date is datetime
field["sample_date"] = pd.to_datetime(field["sample_date"], errors="coerce")

field.head()


,_id,station_id,station_name,station_number,full_station_name,station_type,latitude,longitude,status,county_name,sample_code,sample_date,sample_depth,sample_depth_units,anl_data_type,parameter,fdr_result,fdr_text_result,fdr_date_result,fdr_reporting_limit,uns_name,mth_name,fdr_footnote,result
0,1,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,DissolvedOxygen,9.2,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.2
1,2,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,ElectricalConductance,515.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN,515.0
2,3,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,WaterTemperature,6.7,NaN,NaN,0.1,°C,EPA 170.1 (Field),NaN,6.7
3,4,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,pH,7.3,NaN,NaN,0.1,pH Units,EPA 150.1 (Field),NaN,7.3
4,5,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0268A0006,1968-02-01 08:10:00,1.0,Feet,NaN,DissolvedOxygen,9.7,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.7


## Filter to Selected Water Quality Parameters


In [39]:
params = ["pH", "DissolvedOxygen", "ElectricalConductance"]

field_sub = field[field["parameter"].isin(params)]
field_sub.head()

,_id,station_id,station_name,station_number,full_station_name,station_type,latitude,longitude,status,county_name,sample_code,sample_date,sample_depth,sample_depth_units,anl_data_type,parameter,fdr_result,fdr_text_result,fdr_date_result,fdr_reporting_limit,uns_name,mth_name,fdr_footnote,result
0,1,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,DissolvedOxygen,9.2,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.2
1,2,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,ElectricalConductance,515.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN,515.0
3,4,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,pH,7.3,NaN,NaN,0.1,pH Units,EPA 150.1 (Field),NaN,7.3
4,5,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0268A0006,1968-02-01 08:10:00,1.0,Feet,NaN,DissolvedOxygen,9.7,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.7
5,6,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0268A0006,1968-02-01 08:10:00,1.0,Feet,NaN,ElectricalConductance,720.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN,720.0


## Merge Field Data With Station Metadata


In [40]:
merged = field_sub.merge(stations, on="station_id", how="left")
merged.head()


,_id_x,station_id,station_name_x,station_number_x,full_station_name_x,station_type_x,latitude_x,longitude_x,status,county_name_x,sample_code,sample_date,sample_depth,sample_depth_units,anl_data_type,parameter,fdr_result,fdr_text_result,fdr_date_result,fdr_reporting_limit,uns_name,mth_name,fdr_footnote,result,_id_y,station_name_y,full_station_name_y,station_number_y,station_type_y,latitude_y,longitude_y,county_name_y,sample_count,sample_date_min,sample_date_max
0,1,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,DissolvedOxygen,9.2,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.2,43682.0,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.6203,Alameda,1720.0,1960-10-16T07:25:00,2025-10-21T11:35:00
1,2,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,ElectricalConductance,515.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN,515.0,43682.0,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.6203,Alameda,1720.0,1960-10-16T07:25:00,2025-10-21T11:35:00
2,4,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0168A0001,1968-01-04 07:45:00,1.0,Feet,NaN,pH,7.3,NaN,NaN,0.1,pH Units,EPA 150.1 (Field),NaN,7.3,43682.0,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.6203,Alameda,1720.0,1960-10-16T07:25:00,2025-10-21T11:35:00
3,5,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0268A0006,1968-02-01 08:10:00,1.0,Feet,NaN,DissolvedOxygen,9.7,NaN,NaN,0.2,mg/L,EPA 360.2 (Field),NaN,9.7,43682.0,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.6203,Alameda,1720.0,1960-10-16T07:25:00,2025-10-21T11:35:00
4,6,12,H.O. Banks Headworks,KA000331,Delta P.P. Headworks at H.O. Banks PP,Surface Water,37.8019,-121.6203,"Public, Review Status Unknown",Alameda,OM0268A0006,1968-02-01 08:10:00,1.0,Feet,NaN,ElectricalConductance,720.0,NaN,NaN,1.0,uS/cm,Std Method 2510-B (Field),NaN,720.0,43682.0,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.6203,Alameda,1720.0,1960-10-16T07:25:00,2025-10-21T11:35:00


## Q1: Average Values by Station


In [41]:
station_summary = (
    merged.groupby(["station_id", "parameter"], as_index=False)
    .agg(avg=("result", "mean"), count=("result", "count"))
)

station_summary.head()


,station_id,parameter,avg,count
0,9,DissolvedOxygen,8.774684,158
1,9,pH,7.541829,164
2,10,DissolvedOxygen,8.633507,767
3,10,ElectricalConductance,470.948718,39
4,10,pH,7.787925,800


## Q2: Trend Over Time (Example Station)


In [43]:
merged = merged.dropna(subset=["sample_date"])
merged["year"] = merged["sample_date"].dt.year

do_data = merged[merged["parameter"] == "DissolvedOxygen"]

do_data["station_id"].value_counts().head()


,count
station_id,
12,1054
45935,1024
45929,978
10,767
45938,749


In [45]:
top_station = do_data["station_id"].value_counts().idxmax()
top_station

yearly = (
    do_data[do_data["station_id"] == top_station]
    .groupby("year", as_index=False)
    .agg(avg=("result", "mean"))
)

px.line(yearly, x="year", y="avg", title=f"Dissolved Oxygen Over Time (Station {top_station})")


## Q3: Sampling Effort


In [46]:
samples_by_station = (
    merged.groupby("station_id", as_index=False)
    .agg(count=("result", "count"))
    .sort_values("count", ascending=False)
)

samples_by_station.head()


,station_id,count
3,12,2249
1318,45929,1822
1319,45935,1781
1,10,1606
1317,45927,1447


## Q4: Parameter Correlation


In [48]:
# Make a pivot table so each parameter becomes its own column
pivot = merged.pivot_table(
    index=["station_id", "sample_date"],
    columns="parameter",
    values="result",
    aggfunc="mean"
).reset_index()

# See what parameter columns we actually have after pivot
print("Pivot columns:", pivot.columns.tolist())

# Make sure we only use parameters that exist as columns
params = ["pH", "DissolvedOxygen", "ElectricalConductance"]
existing_params = [p for p in params if p in pivot.columns]

print("Using these parameters for correlation:", existing_params)

corr_matrix = pivot[existing_params].corr()
corr_matrix


Pivot columns: ['station_id', 'sample_date', 'DissolvedOxygen', 'ElectricalConductance', 'pH']
Using these parameters for correlation: ['pH', 'DissolvedOxygen', 'ElectricalConductance']


parameter,pH,DissolvedOxygen,ElectricalConductance
parameter,,,
pH,1.000000,-0.000992,0.570200
DissolvedOxygen,-0.000992,1.000000,-0.053621
ElectricalConductance,0.570200,-0.053621,1.000000


## Conclusion


This notebook used real California water quality data to explore:

- Average values by station  
- Trends over time  
- Sampling effort  
- Basic correlations between parameters  

All analysis used tools from class: filtering, groupby, merging, summary statistics, and basic plotting.
